<a href="https://colab.research.google.com/github/chriskary1/energy-forecasting-prophet/blob/main/notebooks/energy_forecast_prophet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
from prophet import Prophet


!pip install gdown
import gdown

#
file_id = '1FFqfEseizl0nukORBYsS2M297edkm3cw'
url = f'https://drive.google.com/uc?id={file_id}'

output = 'AEP_hourly_1.csv'
gdown.download(url, output, quiet=False)


# read dataframe
df = pd.read_csv('AEP_hourly_1.csv')

#
df['ds'] = pd.to_datetime(df['ds'])
df = df.set_index('ds').resample('D').sum().reset_index()
df.rename(columns={'value': 'y'}, inplace=True)

# model with holidays
m = Prophet()
m.add_country_holidays(country_name='US')

# Fit & Forecast
m.fit(df)
future = m.make_future_dataframe(periods=365)
forecast = m.predict(future)

# view forcast
print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail())


Downloading...
From: https://drive.google.com/uc?id=1FFqfEseizl0nukORBYsS2M297edkm3cw
To: /content/AEP_hourly_1.csv
100%|██████████| 1.42M/1.42M [00:00<00:00, 14.5MB/s]
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmp3qz9a8z0/ce7dh81y.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp3qz9a8z0/wxr14ee4.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: 1
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=21688', 'data', 'file=/tmp/tmp3qz9a8z0/ce7dh81y.json', 'init=/tmp/tmp3qz9a8z0/wxr14ee4.json', 'output', 'file=/tmp/tmp3qz9a8z0/prophet_modelm2cobxre/prophet_model-20250806143817.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
14:38:17 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
14:38:17 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Ch

             ds           yhat     yhat_lower     yhat_upper
2401 2019-07-30  401235.362881  360129.406926  439396.547274
2402 2019-07-31  399501.243575  360920.764117  439953.323109
2403 2019-08-01  399985.581002  361889.859801  438130.645714
2404 2019-08-02  391382.194083  351065.262530  432424.835374
2405 2019-08-03  364355.331518  323316.114910  403109.347602


In [5]:
from prophet.plot import plot_plotly, plot_components_plotly
plot_plotly(m, forecast)



In [6]:
plot_components_plotly(m, forecast)

/usr/local/lib/python3.11/dist-packages/plotly/io/_json.py:560: UserWarning:

Discarding nonzero nanoseconds in conversion.



In [ ]:
# Εξαγωγή πλήρους forecast σε CSV
forecast.to_csv('forecast_results.csv', index=False)

# Λήψη του αρχείου τοπικά (Google Colab)
from google.colab import files
files.download('forecast_results.csv')